<a href="https://colab.research.google.com/github/Pager07/Webtech/blob/master/SVDHomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [0]:
import pandas as pd 
import numpy as np

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


#Loading Book and Rating Table



In [0]:
root_dir = '/content/gdrive/My Drive/My Notes/3rd year/SSA/SSA/Web Technology/Webtech:Homework/BookData'
ratings_df = pd.read_csv(root_dir + '/rate_table.csv', sep=',', error_bad_lines=False,encoding="latin-1")
books_df = pd.read_csv(root_dir + '/books_table.csv',sep=',', error_bad_lines=False,encoding="latin-1")


In [6]:
ratings_df.head()

,userID,ISBN,bookRating
0,276822,0805057706,10
1,277143,0345342968,9
2,277157,0679450408,6
3,277179,0679767800,7
4,277427,0385424736,9


In [7]:
books_df.head()

,ISBN,book_title,genre
0,0395366283,Curious George Goes to the Aquarium,Children's Fiction|General
1,0070434247,China: Its History and Culture,Asia - Civilization|Asian Studies - East Asia ...
2,2253141453,De l'amour et autres dÃ?ÃÂ©mons,Fiction
3,0425171299,Charmed,Romance fiction
4,0440508568,What Einstein Didn't Know : Scientific Answers...,Science|Essays
